# English to Hinglish Generation

Here we are trying to generate Hinglish sentence by English sentence 

In [1]:
import pandas as pd

# dfd = pd.read_csv('/kaggle/input/hinglish-dataset/dev.txt', names=['en', 'hing'], usecols=['en', 'hing'], sep='\t')
# print(dfd.shape)
# dfd = dfd.sample(frac=1, random_state=42)
# dfd = dfd.reset_index(drop=True)
# dfd.head()


df = pd.read_csv('/kaggle/input/enhingdataset/train.txt', names=['en', 'hing'], usecols=['en', 'hing'], sep='\t')

In [2]:
# df1 = pd.read_csv("/kaggle/input/hinglish-dataset/ManualAnalysis.csv")
# df1 = df1[["src", "tgt"]]
# df1 = df1.rename(columns = {"src":"en", "tgt":"hing"})
# df = pd.read_csv('/kaggle/input/hinglish-dataset/train.txt', names=['en', 'hing'], usecols=['en', 'hing'], sep='\t')
# df = df.sample(frac=1, random_state=42)
# df = df.reset_index(drop=True)
# df.head()

In [3]:
df

,en,hing
0,Hindi Milaap From Hyderbad,Hindi Milaap From Hyderbad
1,Pension Fund Managers PFMs,Pension Fund PFMs
2,A new roll of blotting paper has been ordered,blotting paper के new roll का आदेश दिया गया है ।
3,Accelerated Irrigation Benefits Programme,त्वरित Irrigation Benefits Programme
4,So they go deep inside mines,इस लिए वह भूमि के अन्दर गहरे mines है
...,...,...
248325,Color packages by their status,packages उनकी स्थिती के अनुसार रंगें
248326,Disable screensaver when playing,screensaver निष्क्रिय करें
248327,Default list of columns visible in the list view,list दृश्य में दृष्टिगोचर columns की Default list
248328,K3b Video DVD Restrictions,K3b Video DVD Restrictions


In [4]:
# import re
# from unicodedata import normalize

# def clean_text(text):
#     text = text.lower()
#     text = normalize('NFD', text)
#     cleaned_text = ''
#     for char in text:
#         if char.isalpha() or char == ' ':
#             cleaned_text += char
#     return cleaned_text

# df['en'] = df['en'].apply(lambda row: clean_text(row))
# df['hing'] = df['hing'].apply(lambda row: clean_text(row))
# df.head()

import numpy as np
import re
from unicodedata import normalize


def clean_text(text, language='en'):
    if isinstance(text, float) and np.isnan(text):  # Check if text is NaN
        return ''  # Return empty string for NaN values
    text = normalize('NFD', text)
    if language == 'en':
        text = re.sub('[^A-Za-z .\']+', '', text)
    elif language == 'hing': 
        text = re.sub('[^\u0900-\u097F A-Za-z .\']+', '', text)
    return text

def clean_and_prepare_text(text, language='hing'):
    text = '[start] ' + clean_text(text, language=language) + ' [end]'
    return text

# Apply it to your dataframe like this:


df['en'] = df['en'].apply(lambda row: clean_text(row, language='en'))
df['hing'] = df['hing'].apply(lambda row: clean_and_prepare_text(row, language='hing'))
df



,en,hing
0,Hindi Milaap From Hyderbad,[start] Hindi Milaap From Hyderbad [end]
1,Pension Fund Managers PFMs,[start] Pension Fund PFMs [end]
2,A new roll of blotting paper has been ordered,[start] blotting paper के new roll का आदेश दिय...
3,Accelerated Irrigation Benefits Programme,[start] त्वरित Irrigation Benefits Programme [...
4,So they go deep inside mines,[start] इस लिए वह भूमि के अन्दर गहरे mines है ...
...,...,...
248325,Color packages by their status,[start] packages उनकी स्थिती के अनुसार रंगें [...
248326,Disable screensaver when playing,[start] screensaver निष्क्रिय करें [end]
248327,Default list of columns visible in the list view,[start] list दृश्य में दृष्टिगोचर columns की D...
248328,Kb Video DVD Restrictions,[start] Kb Video DVD Restrictions [end]


In [5]:
en = df['en']
hing = df['hing']

en_max_len = max(len(line.split()) for line in en)
hing_max_len = max(len(line.split()) for line in hing)

print(f'Max phrase length (English): {en_max_len}')
print(f'Max phrase length (Hinglish): {hing_max_len}')

sequence_len = max(en_max_len,hing_max_len)


Max phrase length (English): 10
Max phrase length (Hinglish): 114


In [6]:
from collections import defaultdict
import torch
class Tokenizer:
    def __init__(self):
        self.word_index = {}
        self.index_word = {}
    
    def fit_on_texts(self, texts):
        word_freq = defaultdict(int)
        for text in texts:
            for word in text.split():
                word_freq[word] += 1
        self.word_index = {word: i+1 for i, (word, freq) in enumerate(sorted(word_freq.items(), key=lambda x: x[1], reverse=True))}
        self.index_word = {i: word for word, i in self.word_index.items()}

    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            seq = [self.word_index.get(word, 0) for word in text.split()]
            sequences.append(seq)
        return sequences
    
    def sequences_to_texts(self, sequences):
        texts = []
        for sequence in sequences:
            words = [self.index_word.get(idx, "<unk>") for idx in sequence if idx > 0]  # Skip padding
            text = " ".join(words)
            texts.append(text)
        return texts

def pad_sequences(sequences, maxlen, padding='post'):
    max_seq_len = max(len(seq) for seq in sequences)
    padded_seqs = torch.zeros((len(sequences), maxlen), dtype=torch.long)
    for i, seq in enumerate(sequences):
        if padding == 'post':
            padded_seqs[i, :len(seq)] = torch.tensor(seq[:maxlen], dtype=torch.long)
        else:  
            padded_seqs[i, -len(seq):] = torch.tensor(seq[-maxlen:], dtype=torch.long)
    return padded_seqs

en_tokenizer = Tokenizer()
en_tokenizer.fit_on_texts(en)
en_sequences = en_tokenizer.texts_to_sequences(en)
en_x = pad_sequences(en_sequences, maxlen=en_max_len, padding='post')

hing_tokenizer = Tokenizer()
hing_tokenizer.fit_on_texts(hing)
hing_sequences = hing_tokenizer.texts_to_sequences(hing)
hing_y = pad_sequences(hing_sequences, maxlen=hing_max_len, padding='post')

In [7]:
en_vocab_size = len(en_tokenizer.word_index) + 1
hing_vocab_size = len(hing_tokenizer.word_index) + 1

print(f'Vocabulary size (English): {en_vocab_size}')
print(f'Vocabulary size (Hinglish): {hing_vocab_size}')

Vocabulary size (English): 81406
Vocabulary size (Hinglish): 114113


## Train a model

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class Seq2Seq(nn.Module):
    def __init__(self, en_vocab_size, hing_vocab_size, hidden_size):
        super(Seq2Seq, self).__init__()
        self.embedding = nn.Embedding(en_vocab_size, 300)
        self.encoder = nn.LSTM(300, hidden_size)
        self.decoder = nn.LSTM(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, hing_vocab_size)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        embedded = self.embedding(x)
        encoded, (hidden, cell) = self.encoder(embedded)
        decoded = self.dropout(encoded)
        decoded, _ = self.decoder(decoded, (hidden, cell))
        output = self.linear(decoded)
        return output

torch.manual_seed(42)
np.random.seed(42)

hidden_size = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Seq2Seq(en_vocab_size, hing_vocab_size, hidden_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

en_x = torch.tensor(en_x, dtype=torch.long).to(device)
hing_y = torch.tensor(hing_y, dtype=torch.long).to(device)



/tmp/ipykernel_34/4092959309.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  en_x = torch.tensor(en_x, dtype=torch.long).to(device)
/tmp/ipykernel_34/4092959309.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hing_y = torch.tensor(hing_y, dtype=torch.long).to(device)


In [ ]:
# Train the model
num_epochs = 20
batch_size = 16
validation_split = 0.2

accumulation_steps = 4
train_losses = []
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    optimizer.zero_grad()  
    for i in range(0, len(en_x), batch_size):
        inputs = en_x[i:i+batch_size]
        labels = hing_y[i:i+batch_size]

        outputs = model(inputs)
        outputs = outputs.permute(1, 0, 2)  
        outputs = outputs.reshape(-1, hing_vocab_size)  

        labels = labels.permute(1, 0)  
        flat_labels = labels.reshape(-1)

        if outputs.size(0) > flat_labels.size(0):
            outputs = outputs[:flat_labels.size(0), :]
        elif outputs.size(0) < flat_labels.size(0):
            flat_labels = flat_labels[:outputs.size(0)]

        loss = criterion(outputs, flat_labels)
        (loss / accumulation_steps).backward()  
        running_loss += loss.item()

        if (i // batch_size + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    running_loss /= (len(en_x) / batch_size)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss:.4f}')
    train_losses.append(running_loss)

In [ ]:
torch.save(model.state_dict(), 'model.pth')

Plot the per-epoch training and validation accuracy:

In [ ]:
# # Train the model
# # num_epochs = 20
# # batch_size = 16
# # validation_split = 0.2

# # accumulation_steps = 4
# # train_losses = []

# model.eval() 
# for i in range(0, len(en_x)):
#     inputs = en_x[i:i+batch_size]
#     labels = hing_y[i:i+batch_size]

#     outputs = model(inputs)
#     outputs = outputs.permute(1, 0, 2)  
#     outputs = outputs.reshape(-1, hing_vocab_size)  

#     labels = labels.permute(1, 0)  
#     flat_labels = labels.reshape(-1)

#     if outputs.size(0) > flat_labels.size(0):
#         outputs = outputs[:flat_labels.size(0), :]
#     elif outputs.size(0) < flat_labels.size(0):
#         flat_labels = flat_labels[:outputs.size(0)]

#     loss = criterion(outputs, flat_labels)
#     (loss / accumulation_steps).backward()  
#     running_loss += loss.item()

#     if (i // batch_size + 1) % accumulation_steps == 0:
#         optimizer.step()
#         optimizer.zero_grad()

# running_loss /= (len(en_x) / batch_size)
# print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss:.4f}')
# train_losses.append(running_loss)

## Using model to translate text

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Seq2Seq(en_vocab_size, hing_vocab_size, hidden_size = 256).to(device)
model.load_state_dict(torch.load('/kaggle/input/lstmmodel/model.pth'))
# torch.load('/kaggle/input/lstmmodel/model.pth')

<All keys matched successfully>

In [13]:
def translate_text_pytorch(text, model, en_tokenizer, hing_tokenizer, en_max_len, device):
    # Tokenizing the input text
    sequence = en_tokenizer.texts_to_sequences([text])
    padded_sequence = torch.tensor(sequence, dtype=torch.long, device=device)  # Directly create tensor on device
    
    padded_sequence = padded_sequence.to(device).long()
    
    if len(padded_sequence.shape) == 1:
        padded_sequence = padded_sequence.unsqueeze(0)
    
    model.eval()
    with torch.no_grad():
        prediction = model(padded_sequence).squeeze(0)
    
    prediction = prediction.cpu().numpy()  # Move prediction to CPU for NumPy operations
    indexes = [np.argmax(idx) for idx in prediction]
    translated_text = hing_tokenizer.sequences_to_texts([indexes])[0]
    
    return translated_text


df_test = pd.read_csv('/kaggle/input/enhingdataset/test.txt', names=['en', 'hing'], usecols=['en', 'hing'], sep='\t')
df_test['en'] = df_test['en'].apply(lambda row: clean_text(row, language='en'))
df_test['hing'] = df_test['hing'].apply(lambda row: clean_and_prepare_text(row, language='hing'))
en_test = df_test['en']
hing_test = df_test['hing']


texts = list(en_test)
translated = []
for text in texts:
    translated.append(translate_text_pytorch(text, model, en_tokenizer, hing_tokenizer, en_max_len, device))
#     print(f'{text} => {translated}')

In [ ]:
# for i in range(20) :
#     print(f'{translated[i]}')


In [14]:
# # Translating "you are good"
# translated_text = translate_text_pytorch('you are good', model, en_tokenizer, hing_tokenizer, en_max_len, device)
# print(translated_text)

with open('translated.txt', 'w', encoding='utf-8') as file:
    for translate in translated:
        file.write(translate + '\n')

The model isn't that good, as the bleu score for this is around 0.0193 only